In [1]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings - Does not give satisfactory results
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.schema import Document

In [4]:
# Create Langhcain documents for IPL players

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )

In [5]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [6]:
vector_store = Chroma(
    # embedding_function=GoogleGenerativeAIEmbeddings(model='models/embedding-001'),
    embedding_function=OpenAIEmbeddings(),
    persist_directory='chroma_db',
    collection_name='sample'
)

C:\Users\hp\AppData\Local\Temp\ipykernel_15444\851970811.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [7]:
# Add documents
vector_store.add_documents(docs)

['12e49242-16fe-49e4-b6e3-64058368f18b',
 '04630dc6-1c11-4a27-be58-6d35785b315e',
 '9adce6ac-b9ef-4617-aa09-ff46adcd8cbb',
 'fe967a21-2f13-4451-859e-4e5fa8c50318',
 '415d95cf-50a1-44e0-b9d7-dc796a8d547a']

You can merge the creation of vector_store and adding documents in it using .from_documents -

vector_store = Chroma.from_documents(
    documents=docs,
    embedding=OpenAIEmbeddings(),
    persist_directory='chroma_db'
)

In [ ]:
# View documents
vector_store.get(include=['embeddings','documents','metadatas'])
# If you directly use .get() then - embeddings: None is shown because of their large size so use include if you want to see embeddings, but if you used from_documents then you need to pass arguments in get.

{'ids': ['12e49242-16fe-49e4-b6e3-64058368f18b',
  '04630dc6-1c11-4a27-be58-6d35785b315e',
  '9adce6ac-b9ef-4617-aa09-ff46adcd8cbb',
  'fe967a21-2f13-4451-859e-4e5fa8c50318',
  '415d95cf-50a1-44e0-b9d7-dc796a8d547a'],
 'embeddings': array([[-0.00210453, -0.00214285,  0.0268    , ..., -0.01707893,
         -0.00366616,  0.01357884],
        [-0.00268021, -0.00010323,  0.02815653, ..., -0.01501936,
          0.00590092, -0.01164922],
        [ 0.00092799, -0.00476   ,  0.0124662 , ..., -0.01731381,
          0.00075886,  0.00296567],
        [-0.02714536,  0.00885395,  0.02699314, ..., -0.02592762,
          0.00900617, -0.01999116],
        [-0.01810451,  0.01281202,  0.0347942 , ..., -0.03034012,
         -0.00595078,  0.00521716]], shape=(5, 1536)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the m

In [9]:
# Search documents
vector_store.similarity_search(
    query='Who among these are a bowler',
    k=2
)

[Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.')]

In [10]:
query = "Who is known as Captain Cool in IPL?"
results = vector_store.similarity_search(query, k=2)

for res in results:
    print("Content:", res.page_content)
    print("Metadata:", res.metadata)

Content: MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.
Metadata: {'team': 'Chennai Super Kings'}
Content: Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.
Metadata: {'team': 'Mumbai Indians'}


In [12]:
# Search with similarity score
vector_store.similarity_search_with_score(
    query='Who among these are a bowler',
    k=2
)

[(Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.34610098600387573),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.40785717964172363)]

In [13]:
# Meta-data filtering
vector_store.similarity_search_with_score(
    query='',
    filter={'team': 'Chennai Super Kings'}
)

[(Document(metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  0.6488258242607117),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.6566494703292847)]

In [16]:
# Update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id='12e49242-16fe-49e4-b6e3-64058368f18b', document=updated_doc1)

In [18]:
# View docuemnt
vector_store.get(include=['embeddings','documents','metadatas'])

{'ids': ['12e49242-16fe-49e4-b6e3-64058368f18b',
  '04630dc6-1c11-4a27-be58-6d35785b315e',
  '9adce6ac-b9ef-4617-aa09-ff46adcd8cbb',
  'fe967a21-2f13-4451-859e-4e5fa8c50318',
  '415d95cf-50a1-44e0-b9d7-dc796a8d547a'],
 'embeddings': array([[-0.00544442, -0.01907989,  0.00706373, ..., -0.01627786,
         -0.00032134,  0.00724619],
        [-0.00268021, -0.00010323,  0.02815653, ..., -0.01501936,
          0.00590092, -0.01164922],
        [ 0.00092799, -0.00476   ,  0.0124662 , ..., -0.01731381,
          0.00075886,  0.00296567],
        [-0.02714536,  0.00885395,  0.02699314, ..., -0.02592762,
          0.00900617, -0.01999116],
        [-0.01810451,  0.01281202,  0.0347942 , ..., -0.03034012,
         -0.00595078,  0.00521716]], shape=(5, 1536)),
 'documents': ["Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple c

In [19]:
# Delete documents
vector_store.delete(ids=['12e49242-16fe-49e4-b6e3-64058368f18b'])

In [20]:
# View documents
vector_store.get(include=['embeddings','documents','metadatas'])

{'ids': ['04630dc6-1c11-4a27-be58-6d35785b315e',
  '9adce6ac-b9ef-4617-aa09-ff46adcd8cbb',
  'fe967a21-2f13-4451-859e-4e5fa8c50318',
  '415d95cf-50a1-44e0-b9d7-dc796a8d547a'],
 'embeddings': array([[-0.00268021, -0.00010323,  0.02815653, ..., -0.01501936,
          0.00590092, -0.01164922],
        [ 0.00092799, -0.00476   ,  0.0124662 , ..., -0.01731381,
          0.00075886,  0.00296567],
        [-0.02714536,  0.00885395,  0.02699314, ..., -0.02592762,
          0.00900617, -0.01999116],
        [-0.01810451,  0.01281202,  0.0347942 , ..., -0.03034012,
         -0.00595078,  0.00521716]], shape=(4, 1536)),
 'documents': ["Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
  'MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.',
  'Jasprit Bumrah i